In [109]:
import pandas as pd
from collections import Counter
import ast
import numpy as np
import matplotlib.pyplot as plt
import datetime
# from skimage.io import imread
import os
import statistics
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier

In [110]:
train_data = pd.read_csv('final_train_data.csv')

In [111]:
# One-hot encode the categorical columns
train_data = pd.get_dummies(train_data, columns=['Sex'])
train_data = pd.get_dummies(train_data, columns=['Frontal/Lateral'])
train_data = pd.get_dummies(train_data, columns=['AP/PA'])

In [112]:
train_data

,Path,Age,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,...,corner_199,Right_Lung_Intensity,Left_Lung_Intensity,Asymmetry,Sex_Female,Sex_Male,Frontal/Lateral_Frontal,Frontal/Lateral_Lateral,AP/PA_AP,AP/PA_PA
0,CheXpert-v1.0-small/train/patient00001/study1/...,68,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,45.543514,43.238125,0.384764,True,False,True,False,True,False
1,CheXpert-v1.0-small/train/patient00002/study2/...,87,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,...,0.0,110.128293,96.570501,0.596201,True,False,True,False,True,False
2,CheXpert-v1.0-small/train/patient00002/study1/...,83,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,...,0.0,144.185999,141.597298,0.482320,True,False,True,False,True,False
3,CheXpert-v1.0-small/train/patient00002/study1/...,83,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,...,0.0,NaN,NaN,NaN,True,False,False,True,False,False
4,CheXpert-v1.0-small/train/patient00003/study1/...,41,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0.0,74.035784,82.150730,0.417489,False,True,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8465,CheXpert-v1.0-small/train/patient02082/study7/...,23,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,False,True,False,True,False,False
8466,CheXpert-v1.0-small/train/patient02082/study5/...,22,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,70.158675,71.296229,0.230087,False,True,True,False,False,True
8467,CheXpert-v1.0-small/train/patient02082/study5/...,22,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,False,True,False,True,False,False
8468,CheXpert-v1.0-small/train/patient02082/study2/...,21,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,56.908656,59.093095,0.264277,False,True,True,False,True,False


In [158]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer  # Import the imputer
from sklearn.model_selection import train_test_split

def prepare_data(train_data, feature_cols, test_size=0.2, random_state=42, pca=False, pca_components=0.95):
    # Remove rows with any NaN values in the target column
    train_data = train_data.dropna(subset=['Pleural Effusion'])
    X = train_data.drop(['Pleural Effusion', 'Path'], axis=1)[feature_cols]  # assuming all other columns are features
    y = train_data['Pleural Effusion']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Encode target variable
    encoder = LabelEncoder()
    y_train_encoded = encoder.fit_transform(y_train)
    y_test_encoded = encoder.transform(y_test)
    
    # Handle NaN values by imputation
    imputer = SimpleImputer(strategy='mean')  # You can choose another strategy like median
    X_train_imputed = imputer.fit_transform(X_train)
    X_test_imputed = imputer.transform(X_test)

    # Check if PCA should be applied
    if pca:
        # Standardize features before applying PCA
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_imputed)
        X_test_scaled = scaler.transform(X_test_imputed)

        # Apply PCA
        pca = PCA(n_components=pca_components)  # Adjust n_components as needed
        X_train_pca = pca.fit_transform(X_train_scaled)
        X_test_pca = pca.transform(X_test_scaled)

        # Explained variance can be printed or logged if needed
        print("Explained Variance Ratio:", pca.explained_variance_ratio_.sum())

        return X_train_pca, X_test_pca, y_train_encoded, y_test_encoded

    return X_train_imputed, X_test_imputed, y_train_encoded, y_test_encoded

In [147]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

def run_xgboost_experiment(X_train, y_train, X_test, y_test):
    model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', objective='multi:softprob', num_class=3)
    model.fit(X_train, y_train)
    y_prob = model.predict_proba(X_test)
    # Assuming you want to treat the problem as binary for ROC AUC purposes
    auc_score = roc_auc_score(y_test, y_prob, multi_class='ovo')  # One-vs-One strategy for multi-class ROC
    return auc_score

In [148]:
#RESNET

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, ReLU, Add, Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import roc_auc_score

def build_resnet_model(input_shape, num_classes):
    inputs = Input(shape=(input_shape,))
    x = Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    for _ in range(3):  # Increased depth
        x_shortcut = x
        x = Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
        x = BatchNormalization()(x)
        x = Add()([x, x_shortcut])
        x = ReLU()(x)

    x = Dense(32)(x)
    x = ReLU()(x)
    x = Dropout(0.5)(x)  # Regularization to prevent overfitting
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def train_predict_evaluate_resnet(X_train, y_train, X_test, y_test):
    # Replace NaNs with a placeholder and use an indicator for missing values
    for column in X_train.columns:
        X_train[column + '_missing'] = X_train[column].isna().astype(int)
        X_train[column].fillna(-999, inplace=True)  # Placeholder value
    
    for column in X_test.columns:
        X_test[column + '_missing'] = X_test[column].isna().astype(int)
        X_test[column].fillna(-999, inplace=True)  # Placeholder value

    num_features = X_train.shape[1]
    num_classes = len(set(y_train))

    # Build and train the ResNet model
    model = build_resnet_model(num_features, num_classes)
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)
    
    # Predict probabilities on the test set and calculate AUC
    y_prob = model.predict(X_test)
    auc_score = roc_auc_score(y_test, y_prob, multi_class='ovo')

    return auc_score


In [149]:
#CNN

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Activation
from tensorflow.keras.models import Model

def build_cnn_model(input_shape, num_classes):
    inputs = Input(shape=(input_shape,))
    x = Dense(64, activation='relu')(inputs)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

def train_and_evaluate_cnn(X_train, y_train, X_test, y_test, num_classes):
    # Assuming you have defined the number of features from X_train
    input_shape = X_train.shape[1]
    
    # Build the model
    model = build_cnn_model(input_shape, num_classes)
    
    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)
    
    # Predict probabilities on the test set
    y_prob = model.predict(X_test)
    
    # Calculate AUC using one-vs-one strategy if it's multi-class classification
    auc_score = roc_auc_score(y_test, y_prob, multi_class='ovo')
    
    return auc_score




# TRAIN AND TEST WITHOUT CLINICAL FEATURES

In [151]:
# Define the subset of columns for an experiment
# Correct way to drop multiple columns
train_data_no_clinical = train_data.drop(['Pleural Effusion', 'Path','Right_Lung_Intensity', 'Left_Lung_Intensity', 'Asymmetry', ], axis=1)
feature_columns = train_data_no_clinical.columns  # Modify as needed

# Prepare the data
X_train, X_test, y_train, y_test = prepare_data(train_data, feature_columns, pca=False)

# Run XGBoost experiment
xgb_auc = run_xgboost_experiment(X_train, y_train, X_test, y_test)

print("XGBoost AUC:", xgb_auc)
# print("ResNet AUC:", resnet_auc)


/Users/gabizhelyazkova/anaconda3/envs/Base/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [00:33:38] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost AUC: 0.8501649224043445


In [152]:
auc_score = train_predict_evaluate_resnet(X_train, y_train, X_test, y_test)
print("ResNet Model AUC Score:", auc_score)


/var/folders/ph/wdjw3wn10c1frx2v062_8sbh0000gn/T/ipykernel_47396/2310814652.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[column].fillna(-999, inplace=True)  # Placeholder value
/var/folders/ph/wdjw3wn10c1frx2v062_8sbh0000gn/T/ipykernel_47396/2310814652.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are se

Epoch 1/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.4751 - loss: 6.0785 - val_accuracy: 0.7050 - val_loss: 4.2384
Epoch 2/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7263 - loss: 4.0325 - val_accuracy: 0.7751 - val_loss: 3.5473
Epoch 3/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7358 - loss: 3.4686 - val_accuracy: 0.7654 - val_loss: 3.0450
Epoch 4/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7685 - loss: 2.9279 - val_accuracy: 0.7666 - val_loss: 2.5967
Epoch 5/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7704 - loss: 2.4901 - val_accuracy: 0.7751 - val_loss: 2.2149
Epoch 6/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7883 - loss: 2.0919 - val_accuracy: 0.7751 - val_loss: 1.9203
Epoch 7/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7707 - loss: 1.8168 - val_accuracy: 0.7703 - val_loss: 1.6699
Epoch 8/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7537 - loss: 1.6187 - val_accu

In [153]:
#CNN

# Assuming num_classes is known, e.g., 2 for binary classification
auc_score = train_and_evaluate_cnn(X_train, y_train, X_test, y_test, num_classes=3)
print("CNN Model AUC Score:", auc_score)

Epoch 1/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5737 - loss: 24.5299 - val_accuracy: 0.7328 - val_loss: 1.2830
Epoch 2/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6505 - loss: 1.4273 - val_accuracy: 0.7376 - val_loss: 0.8065
Epoch 3/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7105 - loss: 0.9039 - val_accuracy: 0.7582 - val_loss: 0.7142
Epoch 4/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7022 - loss: 0.8308 - val_accuracy: 0.7473 - val_loss: 0.6935
Epoch 5/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7273 - loss: 0.7496 - val_accuracy: 0.7606 - val_loss: 0.6838
Epoch 6/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7353 - loss: 0.7271 - val_accuracy: 0.7533 - val_loss: 0.6569
Epoch 7/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7357 - loss: 0.7081 - val_accuracy: 0.7703 - val_loss: 0.6483
Epoch 8/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7191 - loss: 0.7250 - val_accuracy: 0

# TRAIN AND TEST WITH CLINICAL FEATURES

In [154]:
# Define the subset of columns for an experiment
# Correct way to drop multiple columns
train_data_clinical = train_data.drop(['Pleural Effusion', 'Path'], axis=1)
feature_columns = train_data_clinical.columns  # Modify as needed

# Prepare the data
X_train, X_test, y_train, y_test = prepare_data(train_data, feature_columns, pca=False)

# Run XGBoost experiment
xgb_auc = run_xgboost_experiment(X_train, y_train, X_test, y_test)

# For ResNet, ensure you have the appropriate setup or use another suitable model
# resnet_auc = run_resnet_experiment(X_train, y_train, X_test, y_test)

print("XGBoost AUC:", xgb_auc)
# print("ResNet AUC:", resnet_auc)

/Users/gabizhelyazkova/anaconda3/envs/Base/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [00:34:27] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost AUC: 0.845799542029737


In [155]:
auc_score = train_predict_evaluate_resnet(X_train, y_train, X_test, y_test)
print("ResNet Model AUC Score:", auc_score)

/var/folders/ph/wdjw3wn10c1frx2v062_8sbh0000gn/T/ipykernel_47396/2310814652.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[column].fillna(-999, inplace=True)  # Placeholder value
/var/folders/ph/wdjw3wn10c1frx2v062_8sbh0000gn/T/ipykernel_47396/2310814652.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are se

Epoch 1/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5581 - loss: 5.4276 - val_accuracy: 0.7424 - val_loss: 4.0168
Epoch 2/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7292 - loss: 3.8295 - val_accuracy: 0.7654 - val_loss: 3.2709
Epoch 3/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7688 - loss: 3.1192 - val_accuracy: 0.7690 - val_loss: 2.6641
Epoch 4/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7628 - loss: 2.5596 - val_accuracy: 0.7642 - val_loss: 2.2156
Epoch 5/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7667 - loss: 2.1458 - val_accuracy: 0.7654 - val_loss: 1.8616
Epoch 6/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7778 - loss: 1.7882 - val_accuracy: 0.7654 - val_loss: 1.5928
Epoch 7/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7714 - loss: 1.5371 - val_accuracy: 0.7703 - val_loss: 1.3691
Epoch 8/100
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7636 - loss: 1.3191 - val_accu

In [156]:
#CNN

# Assuming num_classes is known, e.g., 3 for binary classification
auc_score = train_and_evaluate_cnn(X_train, y_train, X_test, y_test, num_classes=3)
print("CNN Model AUC Score:", auc_score)

Epoch 1/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5811 - loss: 41.2523 - val_accuracy: 0.7267 - val_loss: 3.2445
Epoch 2/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6349 - loss: 3.4888 - val_accuracy: 0.7316 - val_loss: 0.9577
Epoch 3/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6640 - loss: 1.0516 - val_accuracy: 0.7291 - val_loss: 0.8260
Epoch 4/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6849 - loss: 0.8564 - val_accuracy: 0.7437 - val_loss: 0.7701
Epoch 5/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6949 - loss: 0.8256 - val_accuracy: 0.7545 - val_loss: 0.7183
Epoch 6/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7000 - loss: 0.7877 - val_accuracy: 0.7533 - val_loss: 0.6911
Epoch 7/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7356 - loss: 0.7305 - val_accuracy: 0.7545 - val_loss: 0.7027
Epoch 8/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7236 - loss: 0.7439 - val_accuracy: 0

# PCA

In [159]:
# Define the subset of columns for an experiment
# Correct way to drop multiple columns
train_data_clinical = train_data.drop(['Pleural Effusion', 'Path'], axis=1)
feature_columns = train_data_clinical.columns  # Modify as needed

# Prepare the data
X_train, X_test, y_train, y_test = prepare_data(train_data, feature_columns, pca=True)

# Run XGBoost experiment
xgb_auc = run_xgboost_experiment(X_train, y_train, X_test, y_test)

# For ResNet, ensure you have the appropriate setup or use another suitable model
# resnet_auc = run_resnet_experiment(X_train, y_train, X_test, y_test)

print("XGBoost AUC:", xgb_auc)
# print("ResNet AUC:", resnet_auc)

Explained Variance Ratio: 0.9500877077221079


/Users/gabizhelyazkova/anaconda3/envs/Base/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [00:36:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost AUC: 0.7293816358295607


In [160]:
auc_score = train_predict_evaluate_resnet(X_train, y_train, X_test, y_test)
print("ResNet Model AUC Score:", auc_score)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [161]:
#CNN

# Assuming num_classes is known, e.g., 3 for binary classification
auc_score = train_and_evaluate_cnn(X_train, y_train, X_test, y_test, num_classes=3)
print("CNN Model AUC Score:", auc_score)

Epoch 1/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4889 - loss: 1.1653 - val_accuracy: 0.6385 - val_loss: 0.8354
Epoch 2/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6581 - loss: 0.8351 - val_accuracy: 0.6493 - val_loss: 0.8098
Epoch 3/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6950 - loss: 0.7187 - val_accuracy: 0.6651 - val_loss: 0.7837
Epoch 4/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7272 - loss: 0.6702 - val_accuracy: 0.6687 - val_loss: 0.7666
Epoch 5/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7600 - loss: 0.6095 - val_accuracy: 0.6711 - val_loss: 0.7733
Epoch 6/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7683 - loss: 0.5866 - val_accuracy: 0.6929 - val_loss: 0.7722
Epoch 7/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7950 - loss: 0.5385 - val_accuracy: 0.6929 - val_loss: 0.7737
Epoch 8/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8121 - loss: 0.4889 - val_accuracy: 0.

In [162]:
# Define the subset of columns for an experiment
# Correct way to drop multiple columns
train_data_no_clinical = train_data.drop(['Pleural Effusion', 'Path','Right_Lung_Intensity', 'Left_Lung_Intensity', 'Asymmetry', ], axis=1)
feature_columns = train_data_no_clinical.columns  # Modify as needed

# Prepare the data
X_train, X_test, y_train, y_test = prepare_data(train_data, feature_columns, pca=True)

# Run XGBoost experiment
xgb_auc = run_xgboost_experiment(X_train, y_train, X_test, y_test)

print("XGBoost AUC:", xgb_auc)
# print("ResNet AUC:", resnet_auc)


Explained Variance Ratio: 0.9503220542862725


/Users/gabizhelyazkova/anaconda3/envs/Base/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [00:37:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost AUC: 0.7185684845087099


In [ ]:
auc_score = train_predict_evaluate_resnet(X_train, y_train, X_test, y_test)
print("ResNet Model AUC Score:", auc_score)

In [163]:
#CNN

# Assuming num_classes is known, e.g., 3 for binary classification
auc_score = train_and_evaluate_cnn(X_train, y_train, X_test, y_test, num_classes=3)
print("CNN Model AUC Score:", auc_score)

Epoch 1/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4642 - loss: 1.2662 - val_accuracy: 0.6385 - val_loss: 0.8775
Epoch 2/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6370 - loss: 0.8546 - val_accuracy: 0.6602 - val_loss: 0.8227
Epoch 3/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6636 - loss: 0.7826 - val_accuracy: 0.6651 - val_loss: 0.7999
Epoch 4/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7010 - loss: 0.7115 - val_accuracy: 0.6735 - val_loss: 0.7861
Epoch 5/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7350 - loss: 0.6378 - val_accuracy: 0.6796 - val_loss: 0.7832
Epoch 6/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7493 - loss: 0.6123 - val_accuracy: 0.6771 - val_loss: 0.7801
Epoch 7/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7579 - loss: 0.5770 - val_accuracy: 0.6856 - val_loss: 0.7890
Epoch 8/50
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7937 - loss: 0.5088 - val_accuracy: 0.